# Cassandra - LJ example

environment setup

* switched to conda mosdef cassandra

`conda create -n cassandra -y -c conda-forge -c mosdef -c omnia 'python=3.7' mbuild foyer openbabel fresnel py3dmol 'nodejs>=10' jupyterlab mosdef_cassandra matplotlib` 

pip installed gmso

`conda activate cassandra`

In [1]:
import mosdef_cassandra as mc
import foyer
import mbuild as mb
import numpy as np

If the cell below fails, you need to add Cassandra to your path

```
export PATH=/Users/$(whoami)/Projects/Cassandra_V1.2/bin:${PATH}
```

In [2]:
mc.utils.detect_cassandra_binaries();

Using the following executables for Cassandra:
Python: /Users/jenny/miniconda3/envs/cassandra/bin/python
library_setup: /Users/jenny/miniconda3/envs/cassandra/bin/library_setup.py
Cassandra: /Users/jenny/miniconda3/envs/cassandra/bin/cassandra.exe


In [3]:
# ff units nm kcal/mol 
# (doesn't matter except that mbuild randomly converts things without telling us :c)
lj_ff = foyer.Forcefield()
lj_ff.loadFile("FF_xmls/LJ.xml")

/Users/jenny/miniconda3/envs/cassandra/lib/python3.7/site-packages/foyer/forcefield.py:457: UserWarning: Non-atomistic element type detected. Creating custom element for 
  'Creating custom element for {}'.format(element))


In [4]:
ljium = mb.Compound(name="C", pos=[0,0,0]) 
# using "C" as name is a hacky way to make foyer work
# but this is LJium not carbon
#ljium.visualize().show()

In [5]:
typed_ljium = lj_ff.apply(
    ljium, assert_bond_params=False, assert_angle_params=False, 
    assert_dihedral_params=False
) 
L = 10 # this gets multiplied in cassandra x10 (nm->A) 
# but length scale is same as FF
box = mb.Box(mins=[-L/2]*3, maxs=[L/2]*3)

/Users/jenny/miniconda3/envs/cassandra/lib/python3.7/site-packages/parmed/openmm/topsystem.py:238: OpenMMWarning: Adding what seems to be Urey-Bradley terms before Angles. This is unexpected, but the parameters will all be present in one form or another.
  'all be present in one form or another.', OpenMMWarning)


In [6]:
%%bash
# Delete output of previous run
rm -rf *.in.* species* *.out* *.inp* *.log

In [7]:
# Create box and species list
box_list = [box]
species_list = [typed_ljium]

# Since we have an occupied box we need to specify
# the number of each species present in the intial config
mols_in_boxes = [[0]]

system = mc.System(box_list, species_list, mols_in_boxes=mols_in_boxes)
moves = mc.Moves("gcmc", species_list)

custom_args = {
    "chemical_potentials": [-4],
    "rcut_min": 0.5,
    "vdw_cutoff": min(box.lengths)/2.1,
    "charge_style": "none",
    #"charge_cutoff": 14.0,
    "coord_freq": 100,
    "prop_freq": 10,
}

mc.run(system, moves, "equilibration", 10000, 300.0, **custom_args)

/Users/jenny/miniconda3/envs/cassandra/lib/python3.7/site-packages/mbuild/formats/cassandramcf.py:178: UserWarning: No bonds found. Cassandra will interpet this as a rigid species
  warnings.warn("No bonds found. Cassandra will interpet "


Using the following executables for Cassandra:
Python: /Users/jenny/miniconda3/envs/cassandra/bin/python
library_setup: /Users/jenny/miniconda3/envs/cassandra/bin/library_setup.py
Cassandra: /Users/jenny/miniconda3/envs/cassandra/bin/cassandra.exe
Generating fragment libraries...
Running Cassandra...


In [12]:
%%bash

#mkdir lj_results
cp -rf *.in.* species* *.out* *.inp* *.log lj_results/

In [9]:
import ex_render_cassandra as erc

In [13]:
xyzfile = "lj_results/gcmc.out.xyz"
boxfile = "lj_results/gcmc.out.H"
traj = erc.make_traj(xyzfile, boxfile)

/Users/jenny/Projects/cassandra_example/ex_render_cassandra.py:104: UserWarning: genfromtxt: Empty input file: "[]"
  data = np.genfromtxt(arr, dtype = d_xyz)


In [ ]:
# slow!
erc.display_movie(erc.render_sphere_frame, traj, gif="lj_results/traj.gif")

![](lj_results/traj.gif)